Below can upload an entire table from a txt or csv file to the Azure SQL Database.
Required fields:
* "UploadToSQL" is the class name for uploading tables to the database.
* tablepath is the path the file is located in.
* tablename is the name of the new table to be created.

In [ ]:
import sys
import os
sys.path.append(os.environ.get("PYTHONPATH"))

import createNewData.data.config as in_config
from createNewData.pypackages.Azure import Azure

AzurePackage = Azure(in_config)

SqlDataCursor = AzurePackage("UploadToSQL",
                              in_config.shapes,
                              in_config.shapesname[0])

Below collect all unique values in a specific column of a database.
Required fields:
* "SelectDistinct" is the class name for collecting the unique values.
* shape_id is the table name.
* [dbo].[shapes] is the name of the new table to be queried.

In [9]:
import sys
import os
sys.path.append(os.environ.get("PYTHONPATH"))

import createNewData.data.config as in_config
from createNewData.pypackages.Azure import Azure

AzurePackage = Azure(in_config)

shapeIds = AzurePackage("SelectDistinct",
                              "shape_id",
                              "[dbo].[shapes]")

In [1]:
import sys
import os
import time
import pymongo
import urllib
start_time = time.time()
sys.path.append(os.environ.get("PYTHONPATH"))

try:
    import createNewData.data.config as in_config
    from createNewData.pypackages.Azure import Azure
    from createNewData.pypackages.urlHandler import UrlHandler
    AzurePackage = Azure(in_config)
    Url = UrlHandler(in_config)
    
except ImportError as e:
    print("Failed to import critical modules for this script.")
    print("Please confirm that files exist in the correct locations.")
    print(e)
shapeData = {}

for each in shapeIds:
    try:
        # Generate the Pandas table of all the Longtitudes and Latitudes
        elevations = AzurePackage("SelectLongLat",
                                "[shape_id],[shape_pt_lat],[shape_pt_lon]",
                                "[dbo].[shapes]",
                                "[shape_id]",
                                each)
        # Generate the Json document for upload to MongoDB
        shapeData[each.replace(".","_")] = Url("mineElevationData",elevations)
    except Exception as e:
        print("Send exception to exception handler.")
    
    try:
        # Upload the Json document to MongoDB
        AzurePackage("UploadToMongo","shapes",shapeData)
    except Exception as e:
        print("Send exception to exception handler.")


    print("%s seconds" % (time.time() - start_time))

